In [19]:
import pandas as pd
from Opt_model import build_model

In [40]:
# Load data from Excel files
file_path = "D:\\3. study at TUD\\SoSe 25\\7. Gurobi\\NERD_case_data.xlsx"

# One of DC/RDC name is "NA" in the Excel file, which is not a valid column name in pandas.
# To avoid issues, we will read the data with na_values set to an empty list and keep_default_na set to False.
# Read the data from the specified sheets
rdc_demand_df = pd.read_excel(file_path, sheet_name="RDC_Demand", na_values=[], keep_default_na=False)
dc_capacity_df = pd.read_excel(file_path, sheet_name="DC_Capacity", na_values=[], keep_default_na=False)
dc_costs_df = pd.read_excel(file_path, sheet_name="DC_Costs", na_values=[], keep_default_na=False)
plant_var_cost_df = pd.read_excel(file_path, sheet_name="Plant_Var_Costs", na_values=[], keep_default_na=False)
inbound_cost_df = pd.read_excel(file_path, sheet_name="IB_Costs", na_values=[], keep_default_na=False)
outbound_cost_df = pd.read_excel(file_path, sheet_name="OB_Costs", na_values=[], keep_default_na=False)

In [41]:
#Check data imported
print(rdc_demand_df, "\n")
print(dc_capacity_df, "\n")
print(dc_costs_df, "\n")
print(plant_var_cost_df, "\n")
print(inbound_cost_df, "\n")
print(outbound_cost_df, "\n")


   RDC  Demand_bl_per_week
0   BO                 450
1   BR                  60
2   CO                  80
3   HA                 130
4   MN                 110
5   NA                 140
6   NH                 140
7   NL                  70
8   PO                 120
9   PR                 310
10  SP                 200
11  WO                 190 

   DC  DC Cap
0  BO    1000
1  NA     500
2  PR    1000
3  SP     500
4  WO    1000 

   DC  Var Cost  Fixed Cost
0  BO      1.50       11000
1  NA      0.95        5000
2  PR      1.05        9000
3  SP      1.10        8000
4  WO      1.12        7000 

  Plant  Var Cost
0   BFP      2.00
1   SCP      0.75 

   DC   BFP   SCP
0  BO  3.40  4.80
1  NA  3.00  5.25
2  PR  4.40  5.12
3  SP  3.04  4.00
4  WO  3.36  4.20 

   DC     BO     BR     CO     HA     MN     NA     NH     NL     PO     PR  \
0  BO   4.40  51.15  37.95  53.90  30.25  20.35  70.40  52.25  34.10  23.10   
1  NA  20.35  35.75  18.15  56.65  11.00   6.60  75.35  62.15  26.4

In [42]:
# convert the DataFrame to a dictionary for Fast, Direct Access to Each Value
rdc_demand = dict(zip(rdc_demand_df['RDC'], rdc_demand_df['Demand_bl_per_week']))
dc_capacity = dict(zip(dc_capacity_df['DC'], dc_capacity_df['DC Cap']))
dc_fixed_cost = dict(zip(dc_costs_df['DC'], dc_costs_df['Fixed Cost']))
dc_var_cost = dict(zip(dc_costs_df['DC'], dc_costs_df['Var Cost']))
plant_var_cost = dict(zip(plant_var_cost_df['Plant'], plant_var_cost_df['Var Cost']))

inbound_cost = {}
for _, row in inbound_cost_df.iterrows():
    dc = row['DC']
    inbound_cost[('BFP', dc)] = row['BFP']
    inbound_cost[('SCP', dc)] = row['SCP']

outbound_cost = {}

for _, row in outbound_cost_df.iterrows():
    dc = row['DC']  # DC from the first column
    for rdc in outbound_cost_df.columns[1:]:  # skip 'DC' column
        outbound_cost[(dc, rdc)] = row[rdc]


In [43]:
#Check correctness of the data
print(outbound_cost[('BO', 'BR')])  # should return 51.15

51.150000000000006


In [44]:
# Set sets
plants = ['BFP', 'SCP']
dcs = list(dc_capacity.keys())
rdcs = list(rdc_demand.keys())

In [48]:
# Check sets
print("Plants: ", plants)
print("DCs: ", dcs)
print("RDCs: ", rdcs)


Plants:  ['BFP', 'SCP']
DCs:  ['BO', 'NA', 'PR', 'SP', 'WO']
RDCs:  ['BO', 'BR', 'CO', 'HA', 'MN', 'NA', 'NH', 'NL', 'PO', 'PR', 'SP', 'WO']


In [46]:
# Build and solve model
model, x, y, z = build_model(plants, dcs, rdcs, rdc_demand, dc_capacity, dc_fixed_cost, dc_var_cost, plant_var_cost, inbound_cost, outbound_cost)
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22 rows, 75 columns and 195 nonzeros
Model fingerprint: 0x84e23342
Variable types: 70 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+01, 5e+02]
Presolve removed 5 rows and 10 columns
Presolve time: 0.02s
Presolved: 17 rows, 65 columns, 125 nonzeros
Variable types: 60 continuous, 5 integer (5 binary)
Found heuristic solution: objective 78524.400000

Root relaxation: objective 6.087260e+04, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

In [55]:
# Print facilicy opening results
if model.status == 2:
    print(f"Total Cost: {model.ObjVal:.2f}")
    for i in dcs:
        if y[i].X > 0.5:
            print(f"Opened DC: {i}")

Total Cost: 68264.50
Opened DC: NA
Opened DC: SP
Opened DC: WO


In [56]:
# Total Cost Breakdown
print(f"Total Cost: {model.ObjVal:.2f}")
fixed_cost_total = sum(dc_fixed_cost[i] * y[i].X for i in dcs)
inbound_cost_total = sum((plant_var_cost[p] + inbound_cost[(p, i)]) * z[p, i].X for p in plants for i in dcs)
outbound_cost_total = sum((dc_var_cost[i] + outbound_cost[(i, j)]) * x[i, j].X for i in dcs for j in rdcs)

print(f"Fixed DC Costs:     {fixed_cost_total:.2f}")
print(f"Inbound Costs:      {inbound_cost_total:.2f}")
print(f"Outbound Costs:     {outbound_cost_total:.2f}")

#Print Inbound Shipments (from Plant to DC)
print("\n Inbound Shipments (Plant → DC):")
for p in plants:
    for i in dcs:
        if z[p, i].X > 0:
            print(f"{p} → {i}: {z[p, i].X:.1f} barrels")

#Print Outbound Shipments (from DC to RDC)
print("\n Outbound Shipments (DC → RDC):")
for i in dcs:
    for j in rdcs:
        if x[i, j].X > 0:
            print(f"{i} → {j}: {x[i, j].X:.1f} barrels")

# Recheck Demand requirements
print("\n RDC Demand Satisfaction:")
for j in rdcs:
    received = sum(x[i, j].X for i in dcs)
    print(f"{j}: Demand = {rdc_demand[j]}, Delivered = {received:.1f}")


Total Cost: 68264.50
Fixed DC Costs:     20000.00
Inbound Costs:      9825.00
Outbound Costs:     38439.50

 Inbound Shipments (Plant → DC):
BFP → NA: 500.0 barrels
SCP → SP: 500.0 barrels
SCP → WO: 1000.0 barrels

 Outbound Shipments (DC → RDC):
NA → BR: 50.0 barrels
NA → CO: 80.0 barrels
NA → MN: 110.0 barrels
NA → NA: 140.0 barrels
NA → PO: 120.0 barrels
SP → HA: 130.0 barrels
SP → NH: 140.0 barrels
SP → NL: 30.0 barrels
SP → SP: 200.0 barrels
WO → BO: 450.0 barrels
WO → BR: 10.0 barrels
WO → NL: 40.0 barrels
WO → PR: 310.0 barrels
WO → WO: 190.0 barrels

 RDC Demand Satisfaction:
BO: Demand = 450, Delivered = 450.0
BR: Demand = 60, Delivered = 60.0
CO: Demand = 80, Delivered = 80.0
HA: Demand = 130, Delivered = 130.0
MN: Demand = 110, Delivered = 110.0
NA: Demand = 140, Delivered = 140.0
NH: Demand = 140, Delivered = 140.0
NL: Demand = 70, Delivered = 70.0
PO: Demand = 120, Delivered = 120.0
PR: Demand = 310, Delivered = 310.0
SP: Demand = 200, Delivered = 200.0
WO: Demand = 190, D